In [1]:
import numpy as np
import pandas as pd
import librosa
import time
import matplotlib.pyplot as plt
import ESC2

import tensorflow as tf
from tensorflow.keras.utils import to_categorical, plot_model
from tensorflow.python.client import device_lib 
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 16299901664646926992
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 2219980313822134361
physical_device_desc: "device: XLA_CPU device"
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 7046801664
locality {
  bus_id: 1
  links {
  }
}
incarnation: 6466868671250054945
physical_device_desc: "device: 0, name: GeForce GTX 1070, pci bus id: 0000:01:00.0, compute capability: 6.1"
, name: "/device:XLA_GPU:0"
device_type: "XLA_GPU"
memory_limit: 17179869184
locality {
}
incarnation: 12055696645321016258
physical_device_desc: "device: XLA_GPU device"
]


## Fold Training

In [2]:
fold_train_loss = []
fold_train_accuracy = []
fold_valid_loss = []
fold_valid_accuracy = []
fold_test = []

In [3]:
# Define training parameters
max_epochs = 50
lr = 0.0001
batch_size = 64
aug_rate=0.15

# Loop over the functions
for i in range(1, 6):
    
    print(f'\n###############################################################')
    print(f'######################## FOLD-{i} #############################')
    print(f'###############################################################')
    # load the fold data
    train_d, train_l, val_d, val_l, test_d, test_l = ESC2.Load_Segments('ESC50', i)
    
    # Generate training and validation dataset
    training_dataset = ESC2.CreateTrainingSet50(train_d, train_l, name=f'train_F{i}', batch_size=batch_size)
    validation_dataset = ESC2.CreateValidationSet(val_d, val_l, name=f'valid_F{i}', batch_size=batch_size)
    
    # Initialize the network
    net = ESC2.PiczakNet50([60, 41, 2])
    loss_f = tf.keras.losses.CategoricalCrossentropy(from_logits=True)
    opt = tf.keras.optimizers.Adam(lr=lr)
    net.compile(optimizer=opt, loss=loss_f, metrics=["accuracy"])
    
    # Train the network
    epoch_loss, epoch_acc, epoch_vl, epoch_va = ESC2.train(net, max_epochs, training_dataset, validation_dataset, aug_rate=aug_rate, verbose=True)
    
    # Append lossed for confidence plot
    fold_train_loss.append(epoch_loss)
    fold_train_accuracy.append(epoch_acc)
    fold_valid_loss.append(epoch_vl)
    fold_valid_accuracy.append(epoch_va)
    


###############################################################
######################## FOLD-1 #############################
###############################################################
Epoch  1: 	 t-loss: 3.574240 	 t-acc: 0.062547 	 v-loss: 3.306976 	 v-acc: 0.109003 	 time: 176.292


KeyboardInterrupt: 

In [ ]:
# Convert lists to array
fold_train_loss = np.asarray(fold_train_loss)
fold_train_accuracy = np.asarray(fold_train_accuracy)
fold_valid_loss = np.asarray(fold_valid_loss)
fold_valid_accuracy = np.asarray(fold_valid_accuracy)

# Compute mean and confidence interval
m_tl = []
m_ta = []
m_vl = []
m_va = []

ci_tl = []
ci_ta = []
ci_vl = []
ci_va = []

for i in range(max_epochs):
    m_tl.append(np.mean(fold_train_loss[:, i]))
    m_ta.append(np.mean(fold_train_accuracy[:, i]))
    m_vl.append(np.mean(fold_valid_loss[:, i]))
    m_va.append(np.mean(fold_valid_accuracy[:, i]))
    
    ci_tl.append(1.96*np.std(fold_train_loss[:, i])/np.mean(fold_train_loss[:, i]))
    ci_ta.append(1.96*np.std(fold_train_accuracy[:, i])/np.mean(fold_train_accuracy[:, i]))
    ci_vl.append(1.96*np.std(fold_valid_loss[:, i])/np.mean(fold_valid_loss[:, i]))
    ci_va.append(1.96*np.std(fold_valid_accuracy[:, i])/np.mean(fold_valid_accuracy[:, i]))

m_tl = np.asarray(m_tl)
m_ta = np.asarray(m_ta)
m_vl = np.asarray(m_vl)
m_va = np.asarray(m_va)

ci_tl = np.asarray(ci_tl)
ci_ta = np.asarray(ci_ta)
ci_vl = np.asarray(ci_vl)
ci_va = np.asarray(ci_va)

In [ ]:
# Plot loss
fig, ax = plt.subplots()
ax.plot(m_tl, label='train loss')
ax.fill_between(np.arange(0, max_epochs), (m_tl-ci_tl), (m_tl+ci_tl), color='b', alpha=.1)
ax.plot(m_vl, label='validation loss')
ax.fill_between(np.arange(0, max_epochs), (m_vl-ci_vl), (m_vl+ci_vl), color='r', alpha=.1)
plt.grid()
plt.legend()
plt.title('Loss Plot')
plt.show()

In [ ]:
# Plot accuracy
fig, ax = plt.subplots()
ax.plot(m_ta, label='train accuracy')
ax.fill_between(np.arange(0, max_epochs), (m_ta-ci_ta), (m_ta+ci_ta), color='b', alpha=.1)
ax.plot(m_va, label='validation accuracy')
ax.fill_between(np.arange(0, max_epochs), (m_va-ci_va), (m_va+ci_va), color='r', alpha=.1)
plt.grid()
plt.legend()
plt.title('Accuracy Plot')
plt.show()

In [ ]:
# Find the best point before overfit on each run

# Retrain the network until that point

# Evaluate on the test set